In [1]:
import sys
import os
os.chdir(os.path.dirname(sys.path[0]))
print('Current working directory:', os.getcwd())

import torch
import numpy as np
from mmcv import Config
from mmcv.runner import get_dist_info, init_dist, load_checkpoint
from mmaction.models import build_model
from mmaction.datasets import build_dataset

Current working directory: /home/ninv/MyProjects/mmaction2-cil


In [2]:
config_file = 'configs/cil/tsm/tsm_r34_1x1x8_25e_ucf101_rgb_task_0.py'
ckpt_file = 'work_dirs/tsm_r34_1x1x8_25e_ucf101_hflip_rgb_task_0/epoch_50.pth'

cfg = Config.fromfile(config_file)
cfg.load_from = ckpt_file

In [3]:
print('training data loader config:')
cfg.data.train

training data loader config:


{'type': 'RawframeDataset',
 'ann_file': 'data/ucf101/cil_annotations/oracle/oracle_task_0_ucf101_train_split_1_rawframes.txt',
 'data_prefix': 'data/ucf101/rawframes/',
 'pipeline': [{'type': 'SampleFrames',
   'clip_len': 1,
   'frame_interval': 1,
   'num_clips': 8},
  {'type': 'RawFrameDecode'},
  {'type': 'Resize', 'scale': (-1, 256)},
  {'type': 'MultiScaleCrop',
   'input_size': 224,
   'scales': (1, 0.875, 0.75, 0.66),
   'random_crop': False,
   'max_wh_scale_gap': 1,
   'num_fixed_crops': 13},
  {'type': 'Resize', 'scale': (224, 224), 'keep_ratio': False},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_bgr': False},
  {'type': 'FormatShape', 'input_format': 'NCHW'},
  {'type': 'Collect', 'keys': ['imgs', 'label'], 'meta_keys': []},
  {'type': 'ToTensor', 'keys': ['imgs', 'label']}]}

In [4]:
print('validataion data loader config:')
cfg.data.val

validataion data loader config:


{'type': 'RawframeDataset',
 'ann_file': 'data/ucf101/cil_annotations/oracle/oracle_task_0_ucf101_val_split_1_rawframes.txt',
 'data_prefix': 'data/ucf101/rawframes/',
 'pipeline': [{'type': 'SampleFrames',
   'clip_len': 1,
   'frame_interval': 1,
   'num_clips': 8,
   'test_mode': True},
  {'type': 'RawFrameDecode'},
  {'type': 'Resize', 'scale': (-1, 256)},
  {'type': 'CenterCrop', 'crop_size': 224},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_bgr': False},
  {'type': 'FormatShape', 'input_format': 'NCHW'},
  {'type': 'Collect', 'keys': ['imgs', 'label'], 'meta_keys': []},
  {'type': 'ToTensor', 'keys': ['imgs']}]}

In [5]:
# copy validation pipeline config to training pipeline config
cfg.data.train.pipeline = cfg.data.val.pipeline
print('training data loader config:')
cfg.data.train

training data loader config:


{'type': 'RawframeDataset',
 'ann_file': 'data/ucf101/cil_annotations/oracle/oracle_task_0_ucf101_train_split_1_rawframes.txt',
 'data_prefix': 'data/ucf101/rawframes/',
 'pipeline': [{'type': 'SampleFrames',
   'clip_len': 1,
   'frame_interval': 1,
   'num_clips': 8,
   'test_mode': True},
  {'type': 'RawFrameDecode'},
  {'type': 'Resize', 'scale': (-1, 256)},
  {'type': 'CenterCrop', 'crop_size': 224},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_bgr': False},
  {'type': 'FormatShape', 'input_format': 'NCHW'},
  {'type': 'Collect', 'keys': ['imgs', 'label'], 'meta_keys': []},
  {'type': 'ToTensor', 'keys': ['imgs']}]}

In [6]:
train_dataset = build_dataset(cfg.data.train)

In [7]:
model = build_model(cfg.model)
_ = load_checkpoint(model, ckpt_file, map_location='cpu')
device = torch.device('cuda')
model = model.to(device).eval()

load checkpoint from torchvision path: torchvision://resnet34


2022-04-18 01:19:28,930 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}


load checkpoint from local path: work_dirs/tsm_r34_1x1x8_25e_ucf101_hflip_rgb_task_0/epoch_50.pth


In [8]:
idx = 2
sample = train_dataset[idx]
sample_info = train_dataset.video_infos[idx]
with torch.no_grad():
    print('load sample from:', sample_info['frame_dir'])
    out = model(sample['imgs'].unsqueeze(dim=0).to(device), return_loss=False)
    print('gt: {}, pred: {}'.format(sample_info['label'], np.argmax(out))) 

load sample from: /home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/HighJump/v_HighJump_g11_c02
gt: 15, pred: 15


In [9]:
print(model)

Recognizer2D(
  (backbone): ResNetTSM(
    (conv1): ConvModule(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): ConvModule(
          (conv): TemporalShift(
            (net): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU(i

In [10]:
from mmaction.core import OutputHook

In [11]:
idx = 2
sample = train_dataset[idx]
sample_info = train_dataset.video_infos[idx]
# with torch.no_grad():
#     print('load sample from:', sample_info['frame_dir'])
#     out = model(sample['imgs'].unsqueeze(dim=0).to(device), return_loss=False)
#     print('gt: {}, pred: {}'.format(sample_info['label'], np.argmax(out))) 


cnn_layer_names = ['backbone.layer1', 'backbone.layer2', 'backbone.layer3', 'backbone.layer4']

with torch.no_grad() as no_grad, OutputHook(model, cnn_layer_names + ['cls_head.avg_pool', 'cls_head'], as_tensor=True) as output_hooks:
    model(sample['imgs'].unsqueeze(dim=0).to(device), return_loss=False)
    f1, f2, f3, f4 = [output_hooks.layer_outputs[layer_name] for layer_name in cnn_layer_names ]
    repr = output_hooks.layer_outputs['cls_head.avg_pool'].flatten(1)
    repr = repr.view(-1, model.cls_head.num_segments, repr.size(1))
    repr_consensus = model.cls_head.consensus(repr).squeeze(1)
    
    cls_score = output_hooks.layer_outputs['cls_head']


print(repr.shape, repr_consensus.shape)
print(f1.shape, f2.shape, f3.shape, f4.shape)
print(cls_score)

torch.Size([1, 8, 512]) torch.Size([1, 512])
torch.Size([8, 64, 56, 56]) torch.Size([8, 128, 28, 28]) torch.Size([8, 256, 14, 14]) torch.Size([8, 512, 7, 7])
tensor([[ 2.6149,  4.6450, -0.8306,  0.0226, -4.5074, -0.1619, -0.5803,  1.9388,
          1.3907, -3.2450, -1.9223,  1.6897, -3.2510,  0.8924, -0.9608, 11.2810,
         -3.4461,  1.3407,  4.6471,  1.6706, -1.9633,  2.1531, -3.7925, -0.0996,
          0.9811, -4.3720, -0.2790, -3.2767,  1.6480,  2.0886,  1.1023, -4.9315,
          1.5720,  0.7747,  1.0945, -2.1771, -1.6550, -0.0255,  1.6175, -2.5643,
          6.4807, -5.8572,  0.0896,  3.4207, -0.3524, -3.9653, -1.0731, -0.9034,
          0.1455,  3.8042, -2.8586]], device='cuda:0')


In [12]:
pred_classidx = torch.argmax(cls_score)
pred_classidx ,sample_info['label'], pred_classidx.item()==sample_info['label']

(tensor(15, device='cuda:0'), 15, True)

In [14]:
train_dataset.video_infos

[{'frame_dir': '/home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/WritingOnBoard/v_WritingOnBoard_g12_c01',
  'total_frames': 101,
  'label': 14},
 {'frame_dir': '/home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/PlayingDaf/v_PlayingDaf_g20_c01',
  'total_frames': 373,
  'label': 9},
 {'frame_dir': '/home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/HighJump/v_HighJump_g11_c02',
  'total_frames': 88,
  'label': 15},
 {'frame_dir': '/home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/Nunchucks/v_Nunchucks_g14_c01',
  'total_frames': 302,
  'label': 3},
 {'frame_dir': '/home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/HeadMassage/v_HeadMassage_g22_c03',
  'total_frames': 149,
  'label': 27},
 {'frame_dir': '/home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/BenchPress/v_BenchPress_g09_c05',
  'total_frames': 63,
  'label': 22},
 {'frame_dir': '/home/ninv/MyProjects/mmaction2-cil/data/ucf101/rawframes/JumpRope/v_JumpRope_g18_c03',
  'total_frame

In [15]:
hasattr(model, 'cls_head.consensus')

False

In [16]:
model.backbone.layer1

Sequential(
  (0): BasicBlock(
    (conv1): ConvModule(
      (conv): TemporalShift(
        (net): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (conv2): ConvModule(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU(inplace=True)
  )
  (1): BasicBlock(
    (conv1): ConvModule(
      (conv): TemporalShift(
        (net): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (conv2): ConvModule(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   